# Code to create a undersampled version of SQuAD

In [ ]:
! pip install git+https://github.com/huggingface/datasets.git

In [ ]:
from datasets import load_dataset, load_metric
import datasets

## Download both the datasets

In [ ]:
dataset_squad = load_dataset("squad_v2")

In [ ]:
dataset_squad

## Under sampling the dataset
We set a threshold of 10,000 samples per each type of question for the train dataset. This number can be varied as a hyperparameter

In [ ]:
import pickle as pkl
import pandas as pd
import numpy as np

In [ ]:
qa_dict = {'what':[] , 'where': [], 'how': [], 'why':[], 'when': [], 'which':[],  'misc': [], 'who' : []}
qa_keys = ['what', 'where', 'how', 'why', 'when', 'which', 'who']

pos_map = {key : i for i, key in enumerate(qa_dict)}
reverse_pos_map = {i : key for i, key in enumerate(qa_dict)}

In [ ]:
def one_hot(question_type):
  one_hot = np.zeros(len(qa_dict))

  one_hot[pos_map[question_type]] = 1

  return one_hot


In [ ]:
df_train_X = pd.DataFrame(dataset_squad['train'])
train_y = []
df_test = pd.DataFrame(dataset_squad['validation'])

In [ ]:
df_train_X.head()

In [ ]:
for i in range(len(df_train_X)):
  question = df_train_X.iloc[i]['question']
  id = df_train_X.iloc[i]['id']
  misc_flag = True
  for key in qa_keys:
    if key in question.lower():
      qa_dict[key].append(id)
      misc_flag = False
      break
  if misc_flag == True:
    qa_dict['misc'].append(id)

In [ ]:
sorted({key : len(qa_dict[key]) for key in qa_dict.keys()}.items(), key = lambda x : x[1] , reverse = True)

## Under sampling major_key and over sampling minor_key

In [ ]:
major_key = ['what', 'where', 'how', 'when', 'which', 'who']
for key in qa_dict:
  if key in major_key:
    np.random.shuffle(qa_dict[key])
    qa_dict[key] = qa_dict[key][:10000]

In [ ]:
minor_key = ['why' , 'misc']

for key in minor_key:
  while len(qa_dict[key]) < 10000:
    selection = np.random.choice(qa_dict[key])
    qa_dict[key].append(selection)

In [ ]:
sorted({key : len(qa_dict[key]) for key in qa_dict.keys()}.items(), key = lambda x : x[1] , reverse = True)

In [ ]:
values_list = []
for key in major_key:
  for value in qa_dict[key]:
    values_list.append(value)
expt = df_train_X['id'].isin(values_list)
df_train_X['temp'] = expt
major_df_train = df_train_X[df_train_X['temp'] == True]
major_df_train = major_df_train.drop('temp' , axis = 1)

In [ ]:
major_df_train.head()

In [ ]:
temp_df = df_train_X[df_train_X['temp'] == False]

In [ ]:
minor_key = ['why' , 'misc']
minor_train_X = []
for key in minor_key:
  for id in qa_dict[key]:
    temp = temp_df.loc[temp_df['id'] == id]
    minor_train_X.append({col : temp[col].item() for col in temp})

minor_df_train= pd.DataFrame(minor_train_X)

In [ ]:
minor_df_train.head()

## Join the over sampled and under sampled categories

In [ ]:
final_df_train = pd.concat([major_df_train , minor_df_train])

In [ ]:
final_df_train = final_df_train.drop('id' , axis = 1)

In [ ]:
temp = np.arange(len(final_df_train))
temp = [str(i) for i in temp]
final_df_train['id'] = temp
final_df_train

In [ ]:
final_df_train = final_df_train.drop('temp' , axis = 1)

In [ ]:
final_dataset = datasets.DatasetDict({'train' : datasets.Dataset.from_pandas(final_df_train) , 'test' : datasets.Dataset.from_pandas(df_test)})

## Save the data

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_rus.pkl', 'wb') as fp:
  pkl.dump(final_dataset, fp)